In [1]:
import pandas as pd
import python_scripts.football_utility_functions as nfl
from IPython.display import display
# import numpy as np
# import glob
# import matplotlib.pyplot as plt
# from scipy.signal import savgol_filter
# from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
# from sklearn.model_selection import LeaveOneOut

In [2]:
draft_data_sharpe = nfl.read_and_normalize_draft_file("../unprocessed_college_data/draft_picks/leesharpe/draft_picks.csv", 'position')
draft_data_taylor = nfl.read_and_normalize_draft_file("../unprocessed_college_data/draft_picks/seanjtaylor/drafts.csv", 'pos')

In [3]:
combine_data_taylor = nfl.read_and_normalize_combine_data('../unprocessed_college_data/draft_picks/seanjtaylor/combines.csv')

In [4]:
player_data = pd.DataFrame()

for i in range(2005, 2014):
    data_for_year = nfl.read_and_normalize_player_data_for_year(i, 'Position')
    player_data = player_data.append(data_for_year)

In [5]:
sharpe_merged_player_data = player_data.merge(draft_data_sharpe, left_on=['name', 'draft_season', 'unit_key'], right_on=['full_name','season','unit_key'])

# left join so we keep also players that weren't drafted
taylor_merged_player_data = player_data.merge(draft_data_taylor, how='left', left_on=['name', 'draft_season', 'unit_key'], right_on=['player', 'season', 'unit_key'])
taylor_merged_player_data = taylor_merged_player_data.merge(combine_data_taylor, how='left', left_on=['draft_season', 'player', 'unit_key'], right_on=['season', 'player', 'unit_key'])

In [6]:
taylor_merged_player_data = taylor_merged_player_data[['Player Code', 'Team Code', 'name', 'unit_key', 'season_x', 'draft_season', 'round', 'pick', 'team', 'age', 'to']]

max_season = taylor_merged_player_data['season_x'].max()
min_season = taylor_merged_player_data['season_x'].min()
player_statistics = nfl.read_player_statistics_for_years(min_season, 2007)
player_statistics = player_statistics.drop(columns=['Game Code'])

In [7]:
aggregated_stats = player_statistics.groupby('Player Code', as_index=False).sum()
aggregated_stats = taylor_merged_player_data.merge(aggregated_stats, how='left', on='Player Code')

In [8]:
dl_data = nfl.get_position_specific_data(aggregated_stats, 'DL')
wr_data = nfl.get_position_specific_data(aggregated_stats, 'WR')
rb_data = nfl.get_position_specific_data(aggregated_stats, 'RB')
lb_data = nfl.get_position_specific_data(aggregated_stats, 'LB')
qb_data = nfl.get_position_specific_data(aggregated_stats, 'QB')
db_data = nfl.get_position_specific_data(aggregated_stats, 'DB')
ol_data = nfl.get_position_specific_data(aggregated_stats, 'OL')
te_data = nfl.get_position_specific_data(aggregated_stats, 'TE')
ki_data = nfl.get_position_specific_data(aggregated_stats, 'K')

dl_data = nfl.drop_position_irrelevant_columns(dl_data, 'DL')
wr_data = nfl.drop_position_irrelevant_columns(wr_data, 'WR')
rb_data = nfl.drop_position_irrelevant_columns(rb_data, 'RB')
lb_data = nfl.drop_position_irrelevant_columns(lb_data, 'LB')
qb_data = nfl.drop_position_irrelevant_columns(qb_data, 'QB')
db_data = nfl.drop_position_irrelevant_columns(db_data, 'DB')
ol_data = nfl.drop_position_irrelevant_columns(ol_data, 'OL')
te_data = nfl.drop_position_irrelevant_columns(te_data, 'TE')
ki_data = nfl.drop_position_irrelevant_columns(ki_data, 'K')

# TODO: Add team statistics for OLiner to have some stats

DL function called
WR function called
RB function called
LB function called
QB function called
DB function called
OL function called
WR function called
Kicker function called


In [12]:
positional_data = {
    'DL': dl_data,
    'WR': wr_data,
    'RB': rb_data,
    'LB': lb_data,
    'QB': qb_data,
    'DB': db_data,
    'OL': ol_data,
    'TE': te_data,
    'KI': ki_data,
}

In [ ]:

wr_data.to_csv(path_or_buf='../processed_data/wr_college_data.csv')

In [10]:
# ml_dataset = pd.DataFrame(positional_data.get('DL'))
# train_data = ml_dataset.query('draft_season==2006')
# test_data = ml_dataset.query('draft_season==2007')
#
# y_train = train_data['pick']
# x_train = train_data.drop(columns=['round', 'pick'])
# y_test = test_data['pick']
# x_test = test_data.drop(columns=['round', 'pick'])



# train_index, test_index = tscv.split(x)
# X_train, X_test = x[train_index], x[test_index]
# y_train, y_test = y[train_index], y[test_index]




In [11]:
# sharpe_shape = sharpe_merged_player_data.shape
# sharpe_counts = sharpe_merged_player_data["draft_season"].value_counts()
# sharpe_merged_2017 = sharpe_merged_player_data.query('draft_season==2007')
# sharpe_merged_3_rounds = sharpe_merged_player_data.query('round<=3')
# sharpe_counts = sharpe_merged_3_rounds["draft_season"].value_counts()
#
# taylor_counts = taylor_merged_player_data["draft_season"].value_counts()
# taylor_merged_2017 = taylor_merged_player_data.query('draft_season==2007')
# taylor_merged_3_rounds = taylor_merged_player_data.query('round<=3')
# taylor_counts = taylor_merged_3_rounds["draft_season"].value_counts()
#
# #%% new_data: Dataframe containing ID to identify player in all cfb files and also name to identify in all draft pick sets
#
# data_by_name_and_position = pd.DataFrame()
# data_by_name_and_position["name"] = player_data["name"]
# data_by_name_and_position["position"] = player_data["Position"]
# data_by_name_and_position["id"] = player_data["Player Code"]
# data_by_name_and_position.drop_duplicates(subset=["name", "position"], inplace=True, keep=False)
#